Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [6]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

In [7]:
from html.parser import HTMLParser
htmlParser = HTMLParser()
combine_df['clean_tweet'] = combine_df['tweet'].apply(lambda x: htmlParser.unescape(x))
combine_df['clean_tweet']

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
49154    thought factory: left-right polarisation! #tru...
49155    feeling like a mermaid ð #hairflip #neverre...
49156    #hillary #campaigned today in #ohio((omg)) & u...
49157    happy, at work conference: right mindset leads...
49158    my   song "so glad" free download!  #shoegaze ...
Name: clean_tweet, Length: 49159, dtype: object

2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [8]:
def clean_user(text):
  return re.sub(r'@[\w]*', ' ', text)

In [9]:
clean_user_vect = np.vectorize(clean_user)
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: clean_user_vect(x))
combine_df.head(10)

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it's so #gr8 !


3. Изменим регистр твитов на нижний с помощью .lower()

In [10]:
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: x.lower())

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [11]:
def apostr(text, _dict):
  new_text = text
  for word in text.split():
    if word in _dict.keys():
      print(word, _dict[word])
      new_text = re.sub(word, _dict[word], text)
      print(text, new_text)
  return new_text

combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: apostr(x, apostrophe_dict))

can't cannot
    thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked     thanks for #lyft credit i cannot use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked
don't do not
    thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked     thanks for #lyft credit i can't use cause they do not offer wheelchair vans in pdx.    #disapointed #getthanked
can't cannot
the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl the next school year is the year for exams.ð¯ cannot think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl
i'm I am
     welcome here !  i'm   it's so #gr8 !       welcome here !  I am   it's so #gr8 ! 
it's it has / it is
     welcome here !  i'm   it's so #gr8 !       welcome here !  i'm   it has / it is so #gr8 !

won't will not
these niggas burning & won't let you know & these females burning & won't let you know   these niggas burning & will not let you know & these females burning & will not let you know  
won't will not
these niggas burning & won't let you know & these females burning & won't let you know   these niggas burning & will not let you know & these females burning & will not let you know  
i'll I shall / I will
#pinoyandproud   indence day...for all the filipino in whole world... #lizquenfanshere need follower ...promise i'll also follow u #pinoyandproud   indence day...for all the filipino in whole world... #lizquenfanshere need follower ...promise I shall / I will also follow u
it'd it had / it would
if i kept getting interrupted & talked over by commercials, it'd piss me off. ð    #livestream #dontwanttomissanyoftheshow    if i kept getting interrupted & talked over by commercials, it had / it would piss me off. ð    #livestream #dontwanttomissanyoftheshow   
it's it has 

smiling is infectious! we've been having fun passing on a smile in #yr2.... why don't you have a go! #smile   ð  smiling is infectious! we have been having fun passing on a smile in #yr2.... why don't you have a go! #smile   ð 
don't do not
smiling is infectious! we've been having fun passing on a smile in #yr2.... why don't you have a go! #smile   ð  smiling is infectious! we've been having fun passing on a smile in #yr2.... why do not you have a go! #smile   ð 
can't cannot
must can't do it i'm ha right mind    must cannot do it i'm ha right mind   
i'm I am
must can't do it i'm ha right mind    must can't do it I am ha right mind   
you'll you shall / you will
you'll never find another me.   #me #myself #selfie #igers_malaysia   #girls #strongâ¦  you shall / you will never find another me.   #me #myself #selfie #igers_malaysia   #girls #strongâ¦ 
i'm I am
bitch i'm crying ð­  bitch I am crying ð­ 
it's it has / it is
   make sure you register for the   fun run! it'

when bron was in south beach all you miami fans loved him and his crying but since he left y'all y'all wanna bash him smmfh   when bron was in south beach all you miami fans loved him and his crying but since he left you all you all wanna bash him smmfh  
it's it has / it is
it's one of those days again.. monday..missing mom a lot..ð¢ramadan..can't have coffee.. missing tammy heaps..can't see her face live. ð   it has / it is one of those days again.. monday..missing mom a lot..ð¢ramadan..can't have coffee.. missing tammy heaps..can't see her face live. ð  
i'm I am
my body is extremely sore. and i'm losing. ð #annoyed   #inpain my body is extremely sore. and I am losing. ð #annoyed   #inpain
that's that has / that is
wait... that's really a priority of yours?   wait... that has / that is really a priority of yours?  
can't cannot
  butthu 5h fans can't take the heat that's why the only come back y'all have is to insult   ðð   butthu 5h fans cannot take the heat t

la tahzan.. innallaha ma'ana... don't be sad, allah always with me...   mondayð la tahzan.. innallaha ma'ana... do not be sad, allah always with me...   mondayð
i've I have
a friend i've loved twenty years proved the true, greedy/unempathic essence of himself today, which i can no longer deny or excuse.   a friend I have loved twenty years proved the true, greedy/unempathic essence of himself today, which i can no longer deny or excuse.  
what's what has / what is
there comes a time in life where you just got to stop caring and just go with what's meant to happen and be at peace with it. #truth   there comes a time in life where you just got to stop caring and just go with what has / what is meant to happen and be at peace with it. #truth  
i'm I am
hot fuss is having a reissue on vinyl tomorrow and i'm so excited and it'll be such an amazing bihday present to myself ðð   hot fuss is having a reissue on vinyl tomorrow and I am so excited and it'll be such an amazing bihday 

...

i gotta say, making/designing this shi was the best thing i've done ð #shi   #meme #tiredâ¦  i gotta say, making/designing this shi was the best thing I have done ð #shi   #meme #tiredâ¦ 
it's it has / it is
aww yeah it's all good bing bong bing bong    aww yeah it has / it is all good bing bong bing bong   
that's that has / that is
iaint never trip that's why i'm where i'm at â¼ï¸   iaint never trip that has / that is why i'm where i'm at â¼ï¸  
i'm I am
iaint never trip that's why i'm where i'm at â¼ï¸   iaint never trip that's why I am where I am at â¼ï¸  
i'm I am
iaint never trip that's why i'm where i'm at â¼ï¸   iaint never trip that's why I am where I am at â¼ï¸  
don't do not
don't worry about what any one else is doing.  focus on what you are doing. #fitness #healthy    do not worry about what any one else is doing.  focus on what you are doing. #fitness #healthy   
can't cannot
  i can't wait to find out who. :-)  !   i cannot wait to find out who. :-)  

i'm I am
happy friday friends! #tgif ð it's here, #weekend is here! ððð± i'm so   and relieved ðâ¦  happy friday friends! #tgif ð it's here, #weekend is here! ððð± I am so   and relieved ðâ¦ 
can't cannot
can't wait 2 see   tomorrow!!!! #justkeepswimming #ilovedorysomuch #doryismyfavoritefish   cannot wait 2 see   tomorrow!!!! #justkeepswimming #ilovedorysomuch #doryismyfavoritefish  
couldn't could not
couldn't decide.  #twofilters,   #feelinggreat #feelingcute #pout #cutie #summer  could not decide.  #twofilters,   #feelinggreat #feelingcute #pout #cutie #summer 
it's it has / it is
â it's vital to address porn's influence when discussing , sexual abuse, and demeaning language about women.  â it has / it is vital to address porn's influence when discussing , sexual abuse, and demeaning language about women. 
she's she has / she is
seriously this #pitbull she's so   to have the #family #home   casa de walsh  seriously this #pitbull she has / she is so

that's that has / that is
if you haven't noticed yourself karma is something that's very real in your own waking life every single day. blinded by dishonesty.   if you haven't noticed yourself karma is something that has / that is very real in your own waking life every single day. blinded by dishonesty.  
it's it has / it is
  this morning a young kid got up from his seat for me. none designated seats got up or even looked at me. it's them not us.     this morning a young kid got up from his seat for me. none designated seats got up or even looked at me. it has / it is them not us.  
i'm I am
      ð i'm over it       ð I am over it
we'll we will
dinner date went really well   easy conversation and we'll meet up again when he's back in town #greatending dinner date went really well   easy conversation and we will meet up again when he's back in town #greatending
he's he has / he is
dinner date went really well   easy conversation and we'll meet up again when he's back in town #g

i love the stone roses, but the two new songs they've put out so far just don't do much for me.   #stoneroses #iwanttobelieve i love the stone roses, but the two new songs they've put out so far just do not do much for me.   #stoneroses #iwanttobelieve
i'm I am
seems unreal that i'm coming home in only 4 weeks and 4 days.   #goingtomissit seems unreal that I am coming home in only 4 weeks and 4 days.   #goingtomissit
can't cannot
if you can't find a way, create one. anon  #leadership #motivation #inspiration  #quote #inspirational   if you cannot find a way, create one. anon  #leadership #motivation #inspiration  #quote #inspirational  
won't will not
the original moto 360 and lg g watch won't be upgraded to android wear 2.0 | the verge     the original moto 360 and lg g watch will not be upgraded to android wear 2.0 | the verge    
don't do not
   we don't need much to be  , just an unlimited supply of #wool and #cotton ð       we do not need much to be  , just an unlimited supply 

   the nba these days has to be the most fixed spo i've ever watched.. why can't they let pure athletics and talent decide gâ¦    the nba these days has to be the most fixed spo i've ever watched.. why cannot they let pure athletics and talent decide gâ¦
doesn't does not
thanks a lot $15 minimum wage idiots now i have to pay 1.19 for a hot in spicy and the dollar menu doesn't exist ð   thanks a lot $15 minimum wage idiots now i have to pay 1.19 for a hot in spicy and the dollar menu does not exist ð  
it's it has / it is
it's all becoming a little bit real now!   #csadegreeshow    it has / it is all becoming a little bit real now!   #csadegreeshow   
i'm I am
absolutely hate it when the show i'm watching kills off one of my favorite characters   absolutely hate it when the show I am watching kills off one of my favorite characters  
it's it has / it is
and yet people still think it's better to have more guns.     and yet people still think it has / it is better to have more gun

In [12]:
combine_df.head(10)

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it has / it is so #...


5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [13]:
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: apostr(x, short_word_dict))
combine_df.head(20)

u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ðð

it is very sad for the ppl of venezuela  it is very sad for the people of venezuela 
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
   #model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦      #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
   #model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦      #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
              no u don't. did u not single out a race ?                no you don't. did you not single oyout a race ? 
u you
              no u don't. d

u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
u you
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦   #model   i love you take with you all the time in yourð±!!! ððððð¦ð¦ð¦  
ppl people
  sadly it has / it is an international situation.. many ppl r losing their lives around the world #shameonhumanity     sadly it has / it is an international situation.. many people r losing their lives around the world #shameonhumanity  
r are


luv love
  father day... . i luv my dad   father day... . i love my dad
ur you are
it's so frustrating when ur so tired and cannot sleeeeeeep when u finally get home like wutttt. #shiit   it's so frustrating when you are so tired and cannot sleeeeeeep when u finally get home like wutttt. #shiit  
u you
it's so frustrating when ur so tired and cannot sleeeeeeep when u finally get home like wutttt. #shiit   it's so fryoustrating when your so tired and cannot sleeeeeeep when you finally get home like wyoutttt. #shiit  
lol laugh out loud
when life throws lemons at your head, go get coffee. lol #coffeetime #chroniclife #autoimmune #fibrowarrior #relief   #ache #body fog when life throws lemons at your head, go get coffee. laugh out loud #coffeetime #chroniclife #autoimmune #fibrowarrior #relief   #ache #body fog
ppl people
  #conormcgregor runs his mouth endlessly like #rondarousey. both have lost. and yet, ppl only take glee in her losing. #â¦   #conormcgregor runs his mouth endlessly li

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it has / it is so #...


6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [14]:
def replaced(message, world_two, word_dict):
  words = message.split()
  new_message = message
  for word in words:
    if word in word_dict.keys():
      new_message = new_message.replace(word, world_two)
  return new_message

combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: replaced(x, ' ', emoticon_dict))


7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [15]:
def punctuation(message):
  return re.sub(r'[^\w\s]',' ', message)
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: punctuation(x))

In [16]:
combine_df.head(10)

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so se...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause ...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,model i love you take with you all the time...
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams ð ...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it has it is so ...


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [17]:
def delete_symbol(message):
  return re.sub(r'[^a-zA-Z0-9]',' ', message)
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: delete_symbol(x))

9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [18]:
def clean_digit(message):
  return re.sub(r'[^a-zA-Z]',' ', message)
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: clean_digit(x))

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [19]:
def clean_word(x):
   return ' '.join([w for w in x.split() if len(w) > 1])
combine_df['clean_tweet'] = combine_df['clean_tweet'].apply(lambda x: clean_word(x))

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [20]:
from nltk import tokenize as tknz
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/janecalm/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
combine_df['tweet_token'] = combine_df['clean_tweet'].apply(lambda x: tknz.word_tokenize(x))
combine_df['tweet_token']

0        [when, father, is, dysfunctional, and, is, so,...
1        [thanks, for, lyft, credit, can, use, cause, t...
2                                  [bihday, your, majesty]
3        [model, love, you, take, with, you, all, the, ...
4                   [factsguide, society, now, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, in, ohio, omg, us...
49157    [happy, at, work, conference, right, mindset, ...
49158    [my, song, so, glad, free, download, shoegaze,...
Name: tweet_token, Length: 49159, dtype: object

12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [22]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/janecalm/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
stop_words = set(stopwords.words("english"))
def clean_stopwords(message, stop_words):
  clean_text = [word for word in message if not word in stop_words]
  return clean_text
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].apply(lambda x: clean_stopwords(x, stop_words))
combine_df['tweet_token_filtered']

0        [father, dysfunctional, selfish, drags, kids, ...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                                [model, love, take, time]
4                        [factsguide, society, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, ohio, omg, used, ...
49157    [happy, work, conference, right, mindset, lead...
49158    [song, glad, free, download, shoegaze, newmusi...
Name: tweet_token_filtered, Length: 49159, dtype: object

13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [24]:
stemp = nltk.stem.PorterStemmer()
def stemmer(message, stemmer):
  return [stemmer.stem(word) for word in message]
combine_df['tweet_stemmed'] = combine_df['tweet_token_filtered'].apply(lambda x: stemmer(x, stemp))
combine_df['tweet_stemmed']

0        [father, dysfunct, selfish, drag, kid, dysfunc...
1        [thank, lyft, credit, use, caus, offer, wheelc...
2                                        [bihday, majesti]
3                                [model, love, take, time]
4                              [factsguid, societi, motiv]
                               ...                        
49154    [thought, factori, left, right, polaris, trump...
49155    [feel, like, mermaid, hairflip, neverreadi, fo...
49156    [hillari, campaign, today, ohio, omg, use, wor...
49157    [happi, work, confer, right, mindset, lead, cu...
49158    [song, glad, free, download, shoegaz, newmus, ...
Name: tweet_stemmed, Length: 49159, dtype: object

14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [25]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/janecalm/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [26]:
def lemmatizer_f(message, lemmatizer):
  return [lemmatizer.lemmatize(word) for word in message]
combine_df['tweet_lemmatized'] = combine_df['tweet_token_filtered'].apply(lambda x: lemmatizer_f(x, lemmatizer))
combine_df['tweet_lemmatized'] 

0        [father, dysfunctional, selfish, drag, kid, dy...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                                [model, love, take, time]
4                        [factsguide, society, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, ohio, omg, used, ...
49157    [happy, work, conference, right, mindset, lead...
49158    [song, glad, free, download, shoegaze, newmusi...
Name: tweet_lemmatized, Length: 49159, dtype: object

15. Сохраним результат предобработки в pickle-файл.

In [27]:
combine_df.to_pickle('clean.pkl')